### Load the Environment

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Vector embeddings
from langchain_community.embeddings import OpenAIEmbeddings


#### Import Data Loader & Load the data

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://docs.smith.langchain.com/user_guide")
data=loader.load()
print(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}, page_content="\n\n\n\n\nLangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-dat

#### Split the data using RecusiveCharacterTextSplitter ,Create Embeddings and Load it ino a vector database

In [3]:
from langchain_objectbox.vectorstores import ObjectBox
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000)
splitted_data=splitter.split_documents(data)

In [5]:
splitted_data

[Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}, page_content='LangSmith User Guide | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingThis is outdated documentation for 🦜️🛠️ LangSmith, which is no longer actively maintained.For up-to-date document

In [6]:
embeddings=OpenAIEmbeddings()


In [8]:
import os
import shutil



def delete_objectbox_store(directory_path):
    try:
        # Check if the directory exists
        if os.path.exists(directory_path):
            # Delete the directory and all its contents recursively
            shutil.rmtree(directory_path)
            print(f"ObjectBox store at {directory_path} has been deleted.")
        else:
            print(f"The directory {directory_path} does not exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Path to your ObjectBox store directory
objectbox_store_path = r'C:\Users\AKU493\OneDrive - Maersk Group\Hackathon-2024\IMRS\rag\objectbox'

# Call the function to delete the store
delete_objectbox_store(objectbox_store_path)

The directory C:\Users\AKU493\OneDrive - Maersk Group\Hackathon-2024\IMRS\rag\objectbox does not exist.


In [9]:
vector_db_legacy=ObjectBox.from_documents(splitted_data,embedding=embeddings,embedding_dimensions=768)
#vector_db_lastest=ObjectBox.from_documents(splitted_data,embedding=embeddings,embedding_dimensions=768)

### Create a RAG Pipeline(define LLM ,Prompts etc)

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [11]:
llm=ChatOpenAI(model="gpt-4o")

#### Old Way of calling QA Chains

In [12]:

prompt_template=hub.pull("rlm/rag-prompt")

qa_chain =RetrievalQA.from_chain_type(llm,
                            retriever=vector_db_legacy.as_retriever(),
                            chain_type_kwargs={'prompt':prompt_template}
                            )

#Test Legacy Chain
question="what is langsmith"
result=qa_chain.invoke({"query":question})

import pprint
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(result["result"])

c:\Users\AKU493\OneDrive - Maersk Group\Hackathon-2024\IMRS\rag\ragenv\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


('LangSmith is a platform for LLM (Large Language Model) application '
 'development, monitoring, and testing. It supports workflows throughout the '
 'application development lifecycle, including monitoring, A/B testing, and '
 'providing a playground for rapid iteration and experimentation. LangSmith '
 'also facilitates a test-driven approach by allowing developers to create and '
 'run tests on their applications.')


#### Latest Way of calling QA Chains

In [13]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import pprint

# See full prompt at https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(vector_db_legacy.as_retriever(), combine_docs_chain)


result=rag_chain.invoke({"input": "what is langsmith ?"})
pp = pprint.PrettyPrinter(indent=5)
pp.pprint(result["answer"])



c:\Users\AKU493\OneDrive - Maersk Group\Hackathon-2024\IMRS\rag\ragenv\lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


('LangSmith is a platform designed for LLM (Language Model) application '
 'development, monitoring, and testing. It supports workflows across different '
 'stages of the application development lifecycle, including monitoring, A/B '
 'testing, creating and running test sets, and providing a playground '
 'environment for rapid iteration and experimentation.')
